In [1]:
import keras
import tensorflow as tf

# Memory Pre-configuration
config = tf.compat.v1.ConfigProto(
    gpu_options=tf.compat.v1.GPUOptions(
        per_process_gpu_memory_fraction=0.9,
        allow_growth = True
    )
    # device_count = {'GPU': 1}
)
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

###  고급 구조 패턴

#### 배치 정규화

**정규화 (normalization)**는 머신 러닝 모델에 주입되는 샘플들을 균일하게 만드는 광범위한 방법입니다.
* 모델이 학습하고 새로운 데이터에 잘 일반화되도록 돕습니다
* 데이터가 정규 분포를 다른다 가정합니다

`nomarlized_data = (data - np.mean(data, axis = ...)) / np.std(data, axis = ...)`

이전 예제에서는 모델에 데이터를 주입하기 전에 정규화했습니다. 하지만 데이터 정규화는 네트워크에서 일어나는 모든 변환 후에도 고려되어야 합니다. 
* Dense나 Conv2D 층에 들어가는 데이터의 평균이 0이고 분산이 1이더라도 출력되는 데이터가 동일한 분포를 가질 것이라고 기대하기 어렵습니다

**배치 정규화 (batch normalization)**은 훈련하는 동안 평균과 분산이 바뀌더라도 이에 적응하여 데이터를 정규화합니다. 
* 훈련 과정에서 사용된 배치 데이터의 평균과 분산에 대한 *지수 이동 평균 (exponential moving average)*을 내부에 유지
* 배치 정규화의 주요 효과는 잔차 연결과 흡사해 그래디언트의 전파를 도와주는 것
* 결국 더 깊은 네트워크를 구성할 수 있음

Batch Normalization 층은 일반적으로 합성곱이나 완전 연결 층 다음에 사용합니다

```python
conv_model.add(layers.Conv2D(32, 3, activation = "relu"))
conv_model.add(layers.BatchNormalization())

dense_model.add(layers.Dense(32, actiation = "relu"))
dense_model.add(layers.BatchNormalization())
```

BatchNormalization 클래스에는 정규화할 특성 축을 지정하는 axis 매개변수가 있습니다. 
* 이 매개변수의 기본값은 입력 텐서의 *마지막 축*을 나타내는 -1 입니다

#### 깊이별 분리 합성곱

**깊이별 분리 합성곱 (depthwise separable convolution) 층**은 Conv2D를 대체하면서 더 가볍고 (모델 파라미터가 적고) 더 빠른 층입니다
* 채널별로 따로따로 공간 방향의 합성곱을 수행
* 그 다음 점별 합성곱 (1x1 합성곱)을 통해 출력 채널을 합침
* *공간 특성의 학습*과 *채널 방향 특성의 학습*을 분리하는 효과를 냄
* 입력에서 공간상 위치는 상관관계가 크지만 채널별로는 매우 독립적이라고 가정한다면 타당함
* 이 방법은 모델 파라미터와 연산의 수를 크게 줄여주기 때문에 작고 더 빠른 모델을 만듬
* 합성곱을 통해 더 효율적으로 표현을 학습하기 떄문에 적은 데이터로도 더 좋은 표현을 학습함
* 제한된 데이터로 작은 모델을 처음부터 훈련시킬때 특히 더 중요

<br></br>
![](../images/7-2-separablecnn.png)
<br></br>

다음은 작은 데이터셋에서 이미지 분류 문제를 위한 가벼운 깊이별 분리 컨브넷을 만드는 예시입니다

In [2]:
from keras.models import Sequential, Model
from keras import layers

height = 64
width = 64
channels = 3
num_classes = 10

model = Sequential()
model.add(layers.SeparableConv2D(32, 3,
                                 activation = "relu",
                                 input_shape = (height, width, channels, )))
model.add(layers.SeparableConv2D(64, 3, activation = "relu"))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation = "relu"))
model.add(layers.SeparableConv2D(128, 3, activation = "relu"))
model.add(layers.MaxPooling2D(2))

model.add(layers.SeparableConv2D(64, 3, activation = "relu"))
model.add(layers.SeparableConv2D(128, 3, activation = "relu"))
model.add(layers.GlobalAveragePooling2D())

model.add(layers.Dense(32, activation = "relu"))
model.add(layers.Dense(num_classes, activation = "softmax"))

model.compile(optimizer = "rmsprop",
              loss = "categorical_crossentropy")

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
separable_conv2d (SeparableC (None, 62, 62, 32)        155       
_________________________________________________________________
separable_conv2d_1 (Separabl (None, 60, 60, 64)        2400      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 30, 30, 64)        0         
_________________________________________________________________
separable_conv2d_2 (Separabl (None, 28, 28, 64)        4736      
_________________________________________________________________
separable_conv2d_3 (Separabl (None, 26, 26, 128)       8896      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 128)       0         
_________________________________________________________________
separable_conv2d_4 (Separabl (None, 11, 11, 64)        9

### 하이퍼파라미터 최적화

딥러닝 모델을 만들때 무작위로 보이는 결정을 많이 해야합니다
* 얼마나 많은 층을 쌓아야 할까요?
* 층마다 얼마나 많은 유닛이나 필터를 두어야 할까요?
* relu 활성화 함수를 사용해야 할까요?
* 어떤 층 뒤에 Batch Normalization을 사용해야 할까요?
* Dropout은 얼마나 사용해야할까요?

이런 구조에 관련된 파라미터를 **하이퍼파라미터**라고 부릅니다.

전형적인 하이퍼파라미터 최적화 과정은 다음과 같습니다
1. 일련의 하이퍼파라미터를 선택합니다
2. 선택된 하이퍼파라미터로 모델을 만듭니다
3. 훈련 데이터에 학습하고 검증 데이터에서 최종 성능을 측정합니다
4. 다음으로 시도할 하이퍼파라미터를 선택합니다
5. 이 과정을 반복합니다
6. 마지막으로 테스트 데이터에서 성능을 측정합니다

주어진 하이퍼파라미터에서 얻은 검증 성능을 사용하여 다음번에 시도할 하이퍼파라미터를 선택하는 알고리즘이 이 과정의 핵심입니다
* 베이지안 최적화 (Bayesian Optimization)
* 유전 알고리즘 (Genetic Algorithms)
* 간단한 랜덤 탐색 (Random Search)
등이 있습니다

### 모델 앙상블 (Ensemble)

**모델 앙상블 (Model Ensemble)**은 가장 좋은 결과를 얻을 수 있는 또 다른 방법입니다. 앙상블은 여러개 다른 모델의 예측을 합쳐서 더 좋은 예측을 만듭니다.

앙상블은 독립적으로 훈련된 다른 종류의 좋은 모델이 각기 다른 장점을 가지고 있다는 가정을 바탕으로 합니다. 
* 각 모델은 예측을 만들기 위해 조금씩 다른 측면을 바라봅니다
* 데이터의 부분 특징입니다
* 각자의 가정 (고유한 모델 구조와 랜덤 가중치 초기화)을 이용하고 각자의 관점으로 이해합니다
* 각 모델은 데이터의 일부분에 맞는 정답을 찾지만 완전한 정답은 아닙니다.
* 이들의 관점을 모으면 데이터를 훨씬 더 정확하게 묘사할 수 있습니다.

분류의 예를 들어보면 분류기 예측을 합치는 가장 쉬운 방법은 추론할때 나온 예측을 평균 내는 것입니다.

```python
preds_a = model_a.predict(x_val)
preds_b = model_b.predict(x_val)
preds_c = model_c.predict(x_val)
preds_d = model_d.predict(x_val)

final_preds = 0.25 * (preds_a + preds_b + preds_c + preds_d)
```

분류기를 앙상블하는 좋은 방법은 검증 데이터에서 학습된 가중치를 사용하여 가중 평균하는 것입니다. 전형적으로 분류기가 좋은수록 높은 가중치를 가지고 나쁜 분류기일수록 낮은 가중치를 갖습니다.

앙상블이 잘 작동하게 만드는 핵심은 *분류기의 다양성*입니다. 모든 모델이 같은 방향으로 편향되어 있다면 앙상블은 동일한 편향을 유지할 것입니다. 모델이 서로 다른 방향으로 편향되어 있다면 편향은 서로 상쇄되고 앙상블이 더 견고하고 정확해집니다.

이런 이유때문에 가능한 최대한 다르면서 좋은 모델을 앙상블해야 합니다. 일반적으로 매우 다른 구조를 가지거나 다른 종류의 머신 러닝 방법을 말합니다.
* 같은 네트워크를 랜덤 초기화를 다르게 하여 여러번 훈련해서 앙상블하는 것은 거의 해볼 가치가 없습니다
* 모델 사이 차이점이 랜덤 초기화와 모델에 주입되는 훈련 데이터의 순서라면 이 앙상블은 다양성이 낮고 하나의 모델보다 아주 조금만 성능이 향상됩니다